In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
import os
import tqdm
from Libraries import Crawler

In [ ]:
# --- CONFIGURATION ---

my_config = {
    # --- Cấu hình đường dẫn và file ---
    "DATA_DIR": "Database",
    
    # --- Cấu hình bộ lọc ---
    "MIN_YEAR": 2020,
    "MIN_WORDS": 200,
    "MAX_WORDS": 1000,
    
    # --- Cấu hình quy trình crawl ---
    "TARGET_ARTICLES_PER_SUBTYPE": 100,
    "MAX_CONCURRENT_WORKERS": 10,
    "VALIDATION_ARTICLES_COUNT": 10,
    
    # --- Cấu hình các chuyên mục cần crawl ---
    "TYPE_DICT": {
        'thoi-su': ['chinh-tri', 'dan-sinh', 'lao-dong-viec-lam', 'giao-thong', 'mekong', 'quy-hy-vong'],
        'goc-nhin': ['binh-luan-nhieu', 'chinh-tri-chinh-sach', 'y-te-suc-khoe', 'kinh-doanh-quan-tri', 'giao-duc-tri-thuc', 'moi-truong', 'van-hoa-doi-song', 'covid-19', 'tac-gia'],
        'the-gioi': ['tu-lieu', 'phan-tich', 'nguoi-viet-5-chau', 'cuoc-song-do-day', 'quan-su'],
        'kinh-doanh': ['net-zero', 'quoc-te', 'doanh-nghiep', 'chung-khoan', 'ebank', 'vi-mo', 'tien-cua-toi', 'hang-hoa'],
        'bat-dong-san': ['chinh-sach', 'thi-truong', 'khong-gian-song', 'tu-van'],
        'khoa-hoc': ['khoa-hoc-trong-nuoc', 'pii-doi-moi-sang-tao', 'tin-tuc', 'phat-minh', 'ung-dung', 'the-gioi-tu-nhien', 'thuong-thuc'],
        'giai-tri': ['gioi-sao', 'sach', 'video', 'phim', 'nhac', 'thoi-trang', 'lam-dep', 'san-khau-my-thuat'],
        'the-thao': ['bong-da', 'du-lieu-bong-da', 'marathon', 'tennis', 'cac-mon-khac', 'hau-truong'],
        'phap-luat': ['ho-so-pha-an', 'tu-van'],
        'giao-duc': ['tin-tuc', 'tuyen-sinh', 'chan-dung', 'du-hoc', 'thao-luan', 'hoc-tieng-anh', 'giao-duc-40'],
        'suc-khoe': ['tin-tuc', 'cac-benh', 'song-khoe', 'vaccine'],
        'doi-song': ['nhip-song', 'to-am', 'bai-hoc-song', 'cooking', 'tieu-dung'],
        'du-lich': ['diem-den', 'am-thuc', 'dau-chan', 'tu-van', 'cam-nang'],
        'so-hoa': ['cong-nghe', 'san-pham', 'blockchain'],
        'xe': ['thi-truong', 'dien-dan'],
        'y-kien': ['thoi-su', 'doi-song']
    }
}

In [ ]:
crawler = Crawler(config=my_config)
crawled_data = crawler.run()

In [ ]:
print(f"\nĐã crawl xong! Số bài báo mới: {len(crawled_data)}")

if crawled_data:
    df = pd.DataFrame(crawled_data)
    display(df.head())